Add directories to PATH

In [1]:
import sys
import os

path_prefix = "/home/kanthoulis/spider/"

training_scripts_dir = path_prefix + "training"
transforms_dir = path_prefix + "transforms"
image_dir = path_prefix + "image"

sys.path.append(training_scripts_dir)
sys.path.append(transforms_dir)
sys.path.append(image_dir)


Dependencies

In [2]:
#Dependencies
import SimpleITK as sitk
import torch 
import torch.nn as nn
import numpy as np
import pathlib
import os
from natsort import natsorted
import json
import time 
from tqdm import tqdm

import tensor_transforms
import mri_slice
import one_hot

Load tensor_data.json

In [3]:
#Load tensor parameters from .json

json_path = "/home/kanthoulis/spider/tensor_data/tensor_data.json"
with open(json_path, 'r') as file:
    data = json.load(file)

# Assign each value to a variable
row_max = data["row_max"]
col_max = data["col_max"]
image_tensor_min = data["image_tensor_min"]
image_tensor_max = data["image_tensor_max"]
label_tensor_min = data["label_tensor_min"]
label_tensor_max = data["label_tensor_max"]
masks_no = data["masks_no"]
masks_array = data["masks_array"]

value_map = one_hot.value_map(masks_no = masks_no, masks_array= masks_array)
print(row_max)
print(col_max)
print(masks_no)

576
576
20


Paths

In [4]:
path_prefix = "/home/kanthoulis/spider/dataset/"

#train_images = pathlib.Path(path_prefix + "train_augmented_image_slices")
#train_labels = pathlib.Path(path_prefix + "train_augmented_label_slices")
train_images = pathlib.Path(path_prefix + "train_image_slices")
train_labels = pathlib.Path(path_prefix + "train_label_slices")
test_images = pathlib.Path(path_prefix + "test_image_slices")
test_images = pathlib.Path(path_prefix + "test_image_slices")
test_labels = pathlib.Path(path_prefix + "test_label_slices")

train_image_numpy = pathlib.Path(path_prefix + "train_image_numpy")
train_label_numpy = pathlib.Path(path_prefix + "train_label_numpy")
test_image_numpy = pathlib.Path(path_prefix + "test_image_numpy")
test_label_numpy =  pathlib.Path(path_prefix + "test_label_numpy")

train_images_sitk_list = os.listdir(train_images)
train_labels_sitk_list = os.listdir(train_labels)
test_images_sitk_list = os.listdir(test_images)
test_labels_sitk_list = os.listdir(test_labels)

train_images_sitk_list = natsorted(train_images_sitk_list)
train_labels_sitk_list = natsorted(train_labels_sitk_list)
test_images_sitk_list = natsorted(test_images_sitk_list)
test_labels_sitk_list = natsorted(test_labels_sitk_list)

print(len(train_images_sitk_list))
print(len(test_images_sitk_list))

11484
3197


In [5]:
# Loop for processing images
for idx in tqdm(range(0, len(train_images_sitk_list)), desc="Processing training images", unit="image"):
    img_path = train_images.joinpath(train_images_sitk_list[idx])
    numpy_filename = train_images_sitk_list[idx].split('.')[0]

    #print("Processing image:", numpy_filename)

    # Load the image and get the image array
    image = mri_slice.Mri_Slice(img_path)
    image_a = image.hu_a

    # Save the processed image tensor
    image_numpy_path = train_image_numpy.joinpath(f"{numpy_filename}.npy")
    np.save(image_numpy_path, image_a)


# Loop for processing labels
for idx in tqdm(range(0, len(train_labels_sitk_list)), desc="Processing training labels", unit="label"):
    label_path = train_labels.joinpath(train_labels_sitk_list[idx])
    numpy_filename = train_labels_sitk_list[idx].split('.')[0]

    #print("Processing label:", numpy_filename)

    # Load the label and get the label array
    label = mri_slice.Mri_Slice(label_path)
    label_a = label.hu_a

    # Save the processed label tensor
    label_numpy_path = train_label_numpy.joinpath(f"{numpy_filename}.npy")
    np.save(label_numpy_path, label_a)



# Test set
for idx in tqdm(range(0, len(test_images_sitk_list)), desc="Processing test images and labels", unit="test"):
    img_path = test_images.joinpath(test_images_sitk_list[idx])
    label_path = test_labels.joinpath(test_labels_sitk_list[idx])

    numpy_filename = test_images_sitk_list[idx].split('.')[0]

    #print("label", numpy_filename)

    image = mri_slice.Mri_Slice(img_path)
    label = mri_slice.Mri_Slice(label_path)

    image_a = image.hu_a
    label_a = label.hu_a

    image_numpy_path = test_image_numpy.joinpath(f"{numpy_filename}.npy")
    label_numpy_path = test_label_numpy.joinpath(f"{numpy_filename}.npy")

    #print(image_numpy_path)
    np.save(image_numpy_path, image_a)
    np.save(label_numpy_path, label_a)


Processing test images and labels: 100%|██████████| 3197/3197 [00:14<00:00, 224.20test/s]
